# Resources
My multi-kernel jupyter docker image: https://hub.docker.com/r/fzinfz/anaconda3/

Opinionated stacks: https://github.com/jupyter/docker-stacks  
multi-user Hub: https://github.com/jupyterhub/jupyterhub  

Run python online  
http://repl.it/languages/Python  
http://repl.it/languages/Python3   
(Note： http://repl.it can run nearly any language)  

News:  
[As of July 2016 the Anaconda Build system is deprecated](https://docs.continuum.io/anaconda-cloud/build)

# Server Setup
http://jupyter-notebook.readthedocs.io/en/latest/public_server.html  

In [ ]:
jupyter notebook --generate-config

vi ~/.jupyter/jupyter_notebook_config.py   # c.NotebookApp.password

jupyter console # start the ZMQ-based console
jupyter console --existing # If no argument specified, guess most recent

# conda env

In [ ]:
anaconda login
anaconda whoami
conda create --name=py27 python=2.7
conda create --name py35 --clone root
conda remove --name py35 --all
conda env export > py35.yml
conda env create -f py35.yml

conda env upload --file=py35.yml

conda info

conda env list

# ipykernel  
http://ipython.readthedocs.io/en/stable/install/kernel_install.html

In [ ]:
jupyter-kernelspec list

In [ ]:
python3 -m pip install ipykernel
python3 -m ipykernel install --user

python2 -m pip install ipykernel
python2 -m ipykernel install --user

# virtualenv

In [ ]:
yum install python34-setuptools
easy_install-3.4 pip

virtualenv ENV  
cd ENV  
source ./bin/activate  
deactivate

# nbconvert

pandoc deb: https://github.com/jgm/pandoc/releases/  
`jupyter nbconvert --to rst notebook*.ipynb`

http://nbsphinx.readthedocs.io/en/readthedocs-theme/usage.html#HTML-Themes

`pip install sphinx_rtd_theme`

In your conf.py file:
```
import sphinx_rtd_theme
html_theme = "sphinx_rtd_theme"
html_theme_path = [sphinx_rtd_theme.get_html_theme_path()]
```

In [ ]:
# Copyright (c) Jupyter Development Team.
from jupyter_core.paths import jupyter_data_dir
import subprocess
import os
import errno
import stat

PEM_FILE = os.path.join(jupyter_data_dir(), 'notebook.pem')

c = get_config()
c.NotebookApp.ip = os.getenv('INTERFACE', '') or '*'
c.NotebookApp.port = int(os.getenv('PORT', '') or 8888)
c.NotebookApp.open_browser = False

# Set a certificate if USE_HTTPS is set to any value
if 'USE_HTTPS' in os.environ:
    if not os.path.isfile(PEM_FILE):
        # Ensure PEM_FILE directory exists
        dir_name = os.path.dirname(PEM_FILE)
        try:
            os.makedirs(dir_name)
        except OSError as exc: # Python >2.5
            if exc.errno == errno.EEXIST and os.path.isdir(dir_name):
                pass
            else: raise
        # Generate a certificate if one doesn't exist on disk
        subprocess.check_call(['openssl', 'req', '-new', 
            '-newkey', 'rsa:2048', '-days', '365', '-nodes', '-x509',
            '-subj', '/C=XX/ST=XX/L=XX/O=generated/CN=generated',
            '-keyout', PEM_FILE, '-out', PEM_FILE])
        # Restrict access to PEM_FILE
        os.chmod(PEM_FILE, stat.S_IRUSR | stat.S_IWUSR)
    c.NotebookApp.certfile = PEM_FILE

# Set a password if PASSWORD is set
if 'PASSWORD' in os.environ:
    from IPython.lib import passwd
    c.NotebookApp.password = passwd(os.environ['PASSWORD'])
    del os.environ['PASSWORD']

In [2]:
cat ~/.jupyter/jupyter_notebook_config.py

# Configuration file for jupyter-notebook.

#------------------------------------------------------------------------------
# Configurable configuration
#------------------------------------------------------------------------------

#------------------------------------------------------------------------------
# LoggingConfigurable configuration
#------------------------------------------------------------------------------

# A parent class for Configurables that log.
# 
# Subclasses have a log trait, and the default behavior is to get the logger
# from the currently running Application.

#------------------------------------------------------------------------------
# SingletonConfigurable configuration
#------------------------------------------------------------------------------

# A configurable that only allows one instance.
# 
# This class is for classes that should only have one instance of itself or
# *any* subclass. To create and retrieve such a class use the
# :meth:`Sing

In [ ]:
---

# Manually HTTPS + Password setup
```
echo 'password: jupyterpwd'
echo pwd=sha1:189ab089e50b:56e8bf072e6a2df681337bb631c6d5206dd5bd59 > jupyter_env.conf
export cert_folder=/path/to/https/certificates/
docker run --name nb -p 443:8888 --env-file jupyter_env.conf -v $cert_folder:/root/cert/  -v ~:/root/host -it fzinfz/anaconda3 /bin/bash
```
### run jupyter in container
```
jupyter notebook --ip=* --NotebookApp.password=$pwd --certfile=/root/cert/cert1.pem --keyfile=/root/cert/privkey1.pem
```
detach: CTRL-p + CTRL-q
### generate your own password
```
from notebook.auth.security import *
passwd('jupyterpwd')
passwd_check('sha1:189ab089e50b:56e8bf072e6a2df681337bb631c6d5206dd5bd59','jupyterpwd')
```